# Experiment 002: Constructive Heuristic (Well-Aligned Initial Solution)

Implement the zaburo kernel's constructive heuristic that creates a grid-based initial solution with alternating tree orientations. This should achieve score ~88.33, a massive improvement from the sample submission's ~173.

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set precision for Decimal
getcontext().prec = 25
scale_factor = Decimal('1e15')  # Use high precision like zaburo kernel

print('Libraries loaded successfully')

Libraries loaded successfully


In [2]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
def find_best_trees(n: int) -> tuple:
    """Find the best grid-based arrangement for n trees.
    
    Tries different row configurations (n_even, n_odd) and keeps the best.
    Even rows have trees at angle 0, odd rows at angle 180.
    Odd rows are offset by 0.35 to interlock with even rows.
    """
    best_score, best_trees = float("inf"), None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
                
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                # Determine how many trees in this row
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                # Even rows: angle 0, no x offset
                # Odd rows: angle 180, offset by 0.35 (half of 0.7)
                angle = 0 if r % 2 == 0 else 180
                x_offset = Decimal('0') if r % 2 == 0 else Decimal('0.7') / 2
                
                # Y position: even rows at y = r//2 * 1.0
                # Odd rows at y = 0.8 + (r-1)//2 * 1.0
                if r % 2 == 0:
                    y = Decimal(r // 2) * Decimal('1.0')
                else:
                    y = Decimal('0.8') + Decimal((r - 1) // 2) * Decimal('1.0')
                
                # Create trees in this row
                row_trees = [
                    ChristmasTree(
                        center_x=Decimal('0.7') * Decimal(i) + x_offset, 
                        center_y=y, 
                        angle=angle
                    ) 
                    for i in range(m)
                ]
                all_trees.extend(row_trees)
                r += 1
            
            if len(all_trees) != n:
                continue
            
            # Calculate bounding box and score
            xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in all_trees])
            min_x, min_y = xys.min(axis=0)
            max_x, max_y = xys.max(axis=0)
            
            side = max(max_x - min_x, max_y - min_y)
            score = side ** 2
            
            if score < best_score:
                best_score = score
                best_trees = all_trees
    
    return best_score, best_trees

print('find_best_trees function defined')

find_best_trees function defined


In [4]:
# Test with small n
print('Testing with n=10...')
score_10, trees_10 = find_best_trees(10)
print(f'n=10: score = {score_10:.6f}, num_trees = {len(trees_10)}')

print('\nTesting with n=100...')
score_100, trees_100 = find_best_trees(100)
print(f'n=100: score = {score_100:.6f}, num_trees = {len(trees_100)}')

Testing with n=10...
n=10: score = 4.840000, num_trees = 10

Testing with n=100...


n=100: score = 39.690000, num_trees = 100


In [5]:
# Generate solutions for all n from 1 to 200
print('Generating solutions for n=1 to 200...')

solutions = []
for n in tqdm(range(1, 201)):
    score, trees = find_best_trees(n)
    solutions.append((score, trees))

print('Done!')

Generating solutions for n=1 to 200...


  0%|          | 0/200 [00:00<?, ?it/s]

  6%|▌         | 11/200 [00:00<00:02, 88.27it/s]

 10%|█         | 20/200 [00:00<00:07, 25.30it/s]

 12%|█▎        | 25/200 [00:01<00:11, 15.23it/s]

...

 99%|█████████▉| 198/200 [10:16<00:18,  9.11s/it]

100%|█████████▉| 199/200 [10:25<00:09,  9.21s/it]

100%|██████████| 200/200 [10:35<00:00,  9.31s/it]

100%|██████████| 200/200 [10:35<00:00,  3.18s/it]

Done!


In [ ]:
# Calculate overall score
overall_score = sum(score / n for n, (score, _) in enumerate(solutions, 1))
print(f'\nOverall score (constructive heuristic): {overall_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {overall_score - 68.922808:.6f}')

In [ ]:
# Validation functions
def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

# Validate all solutions
print('Validating solutions for overlaps...')
overlap_count = 0
for n, (score, trees) in enumerate(solutions, 1):
    if has_overlap(trees):
        print(f'  Overlap detected in n={n}')
        overlap_count += 1

if overlap_count == 0:
    print('All solutions are valid (no overlaps)!')
else:
    print(f'Found {overlap_count} configurations with overlaps')

In [ ]:
# Fix direction optimization functions
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])


def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    # Use convex hull for efficiency
    hull_points = points_np[ConvexHull(points_np).vertices]
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999), 
        method='bounded'
    )
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-8
    
    if improvement > EPSILON:
        return found_side, found_angle_deg
    else:
        return initial_side, 0.0


def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    # Get bounding box center
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points) / float(scale_factor)
    
    min_xy = points_np.min(axis=0)
    max_xy = points_np.max(axis=0)
    rotation_center = (min_xy + max_xy) / 2.0
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print('Fix direction functions defined')

In [ ]:
# Apply fix_direction to all solutions
print('Applying fix_direction optimization...')

optimized_solutions = []
improved_count = 0

for n, (score, trees) in enumerate(tqdm(solutions), 1):
    if len(trees) < 2:
        optimized_solutions.append((score, trees))
        continue
    
    # Find optimal rotation
    new_side, best_angle = optimize_rotation(trees)
    new_score = new_side ** 2
    
    if new_score < score - 1e-8:
        # Apply rotation
        rotated_trees = apply_rotation(trees, best_angle)
        optimized_solutions.append((new_score, rotated_trees))
        improved_count += 1
    else:
        optimized_solutions.append((score, trees))

print(f'\nImproved {improved_count} configurations')

In [ ]:
# Calculate final score after fix_direction
final_score = sum(score / n for n, (score, _) in enumerate(optimized_solutions, 1))
print(f'\nFinal score after fix_direction: {final_score:.6f}')
print(f'Initial score (constructive): {overall_score:.6f}')
print(f'Improvement: {overall_score - final_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap to target: {final_score - 68.922808:.6f}')

In [ ]:
# Validate optimized solutions
print('Validating optimized solutions for overlaps...')
overlap_count = 0
for n, (score, trees) in enumerate(optimized_solutions, 1):
    if has_overlap(trees):
        print(f'  Overlap detected in n={n}')
        overlap_count += 1

if overlap_count == 0:
    print('All optimized solutions are valid (no overlaps)!')
else:
    print(f'Found {overlap_count} configurations with overlaps')

In [ ]:
# Save submission
def to_str(x):
    return f"s{round(float(x), 6)}"

rows = []
for n, (_, all_trees) in enumerate(optimized_solutions, 1):
    assert len(all_trees) == n, f"Expected {n} trees, got {len(all_trees)}"
    for i_t, tree in enumerate(all_trees):
        rows.append({
            "id": f"{n:03d}_{i_t}",
            "x": to_str(tree.center_x),
            "y": to_str(tree.center_y),
            "deg": to_str(tree.angle),
        })

df = pd.DataFrame(rows)
df.to_csv('/home/code/experiments/002_constructive_heuristic/submission.csv', index=False)
print(f'Saved submission with {len(rows)} rows')

In [ ]:
# Copy to submission folder
import shutil
shutil.copy(
    '/home/code/experiments/002_constructive_heuristic/submission.csv',
    '/home/submission/submission.csv'
)
print('Submission saved to /home/submission/submission.csv')

# Summary
print(f'\n=== SUMMARY ===')
print(f'Constructive heuristic score: {overall_score:.6f}')
print(f'After fix_direction: {final_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {final_score - 68.922808:.6f}')